In [ ]:
!wget -O deskdrop.zip https://github.com/sparsh-ai/reco-data/blob/master/DeskDrop.zip?raw=true
!unzip deskdrop.zip

--2021-05-29 17:54:53--  https://github.com/sparsh-ai/reco-data/blob/master/DeskDrop.zip?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/sparsh-ai/reco-data/raw/master/DeskDrop.zip [following]
--2021-05-29 17:54:53--  https://github.com/sparsh-ai/reco-data/raw/master/DeskDrop.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sparsh-ai/reco-data/master/DeskDrop.zip [following]
--2021-05-29 17:54:53--  https://raw.githubusercontent.com/sparsh-ai/reco-data/master/DeskDrop.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response..

In [ ]:
import math
import pandas as pd

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
article_df = pd.read_csv('shared_articles.csv')
article_df.shape

(3122, 13)

In [ ]:
article_df.sample(5)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
2611,1476980365,CONTENT SHARED,6195056361889291497,3891637997717104548,-6330842961578009837,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,HTML,https://www.palantir.net/blog/addressing-acces...,Addressing Accessibility for the Web,Michellanne Li Making sure your website is acc...,en
2874,1481740935,CONTENT REMOVED,8289800212949675494,-8241940599580729220,-4005961876543065182,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,MG,BR,HTML,http://making.duolingo.com/which-countries-stu...,"Which countries study which languages, and wha...",To find out which languages are the most popul...,en
213,1459868729,CONTENT SHARED,-42126282997118624,2901546026085255870,7996851524082570025,NaN,NaN,NaN,HTML,http://g1.globo.com/am/amazonas/noticia/2016/0...,Programa de Demissão Voluntária na Moto Honda ...,"O Programa de Demissão Voluntária (PDV), impla...",pt
1359,1465736425,CONTENT SHARED,1862503310075246782,3915038251784681624,6469413037900917152,NaN,NaN,NaN,HTML,http://cio.com.br/carreira/2016/06/10/cinco-co...,Cinco competências essenciais ao IT Leaders - CIO,Quanto mais a empresa considera a TI como uma ...,pt
209,1459867824,CONTENT SHARED,-6499865726739947794,6003902177042843076,9221043291398421991,NaN,NaN,NaN,HTML,http://techcrunch.com/2016/04/04/microsofts-mo...,Microsoft's mobile problem may not be a proble...,When Microsoft announced its Windows 10 strate...,en


In [ ]:
interactions_df = pd.read_csv('users_interactions.csv')
interactions_df.shape

(72312, 8)

In [ ]:
interactions_df.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [ ]:
# users can interact with aticles multiple times
interactions_df.groupby(['personId', 'contentId']).size()

personId              contentId           
-9223121837663643404  -8949113594875411859    1
                      -8377626164558006982    1
                      -8208801367848627943    1
                      -8187220755213888616    1
                      -7423191370472335463    8
                                             ..
 9210530975708218054   8477804012624580461    4
                       8526042588044002101    1
                       8856169137131817223    1
                       8869347744613364434    1
                       9209886322932807692    1
Length: 40710, dtype: int64

In [ ]:
interactions_df[(interactions_df['personId']==9210530975708218054)].sort_values(by=['timestamp'])

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
27392,1459248179,VIEW,1805789466376069146,9210530975708218054,5721521926550839425,Android - Native Mobile App,SP,BR
21341,1461270635,VIEW,-5781461435447152359,9210530975708218054,-58622632141160676,Android - Native Mobile App,SP,BR
21354,1461270732,VIEW,-5781461435447152359,9210530975708218054,-58622632141160676,Mozilla/5.0 (Linux; Android 4.4.2; ASUS_T00J B...,SP,BR
21342,1461271628,VIEW,-6346370309134074019,9210530975708218054,-58622632141160676,Mozilla/5.0 (Linux; Android 4.4.2; ASUS_T00J B...,SP,BR
20476,1462462679,VIEW,1024145536490998555,9210530975708218054,6243195827392785882,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
...,...,...,...,...,...,...,...,...
68574,1487258839,VIEW,-14569272361926584,9210530975708218054,-5568826740776362449,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
70612,1487339394,VIEW,-14569272361926584,9210530975708218054,6796634667123705309,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR
70606,1487339645,VIEW,-14569272361926584,9210530975708218054,6796634667123705309,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR
70608,1487339710,VIEW,-14569272361926584,9210530975708218054,6796634667123705309,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR


In [ ]:
interactions_df[(interactions_df['personId']==-9223121837663643404) & (interactions_df['contentId']==-7423191370472335463)].sort_values(by=['timestamp'])

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
60988,1478687166,VIEW,-7423191370472335463,-9223121837663643404,2411828781515959462,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
64003,1478705793,VIEW,-7423191370472335463,-9223121837663643404,2411828781515959462,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
61256,1478795892,VIEW,-7423191370472335463,-9223121837663643404,-4096620279830729916,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
63155,1478860184,VIEW,-7423191370472335463,-9223121837663643404,844431470588928965,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
63183,1478860897,VIEW,-7423191370472335463,-9223121837663643404,844431470588928965,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
63180,1478862151,VIEW,-7423191370472335463,-9223121837663643404,844431470588928965,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
65167,1479290925,VIEW,-7423191370472335463,-9223121837663643404,-8865295409778928525,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
63834,1479376578,VIEW,-7423191370472335463,-9223121837663643404,-6084152871682607649,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR


In [ ]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

print('# total users: {}'.format(interactions_df['personId'].nunique()))
print('# users with at least 5 interactions: {}'.format(len(users_with_enough_interactions_df)))

# total users: 1895
# users with at least 5 interactions: 1140


In [ ]:
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df,how='right',left_on='personId',right_on='personId')

print('# of interactions from users with at least 5 interactions: {}'.format(len(interactions_from_selected_users_df)))

# of interactions from users with at least 5 interactions: 69868


In [ ]:
def preprocess_interactions(data):

  # define implicit signal strengths
  event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,
   }

   # quantify implicit signals based on corresponding strength
   data['eventStrength'] = data['eventType'].apply(lambda x: event_type_strength[x])

   # select users with at least n interactions
   _userids = data.groupby(['personId', 'contentId']).size().groupby('personId').size()
   _userids = _userids[_userids >= 5].reset_index()[['personId']]
  data = data.merge(_userids, how='right', on='personId')

  # apply log transformation to smooth the distribution
  data = data.groupby(['personId', 'contentId'])['eventStrength'].sum().apply(lambda x: math.log(1+x, 2)).reset_index()

In [ ]:
stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

In [ ]:
def preprocess_items(data):

  # get only eventtype='CONTENT SHARED'
  data = data[data['eventType']=='CONTENT SHARED']
  
  # convert 'title' and 'text' column texts into tf-idf based numeric vectors
  tfidf_matrix = vectorizer.fit_transform(data['title'] + "" + data['text'])

source - https://nbviewer.jupyter.org/github/amrqura/end-to-end-recommender-system/blob/main/notebook/recommenderSystemNotebook.ipynb